# Upperair Obs with Contours

By: Kevin Goebbert

An example using the declarative syntax to plot upperair observations and overlay contours.

In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
from metpy.io import add_station_lat_lon
from metpy.plots.declarative import *
from metpy.units import units
import numpy as np
from siphon.simplewebservice.iastate import IAStateUpperAir
import xarray as xr

## Get Upperair Observations

Using the functionality from Siphon to get all of the upperair data from the Iowa State archive. This example uses the current time and determines the date for yesterday to make sure data is available. This data doesn't have lat/lon information, so we add it using the functionality from MetPy to add that information to a dataFrame.

In [ ]:
# Set the date for data and plot
yesterday = datetime.utcnow() - timedelta(days=1)
date = datetime(yesterday.year, yesterday.month, yesterday.day, 12)

# Request data using Siphon request for data from Iowa State Archive
data = IAStateUpperAir.request_all_data(date)

# Add lat/lon information to dataframe
data = add_station_lat_lon(data, data.station.name)

## Get GFS data

For plotting contours we need some gridded output. Using the UCAR THREDDS data server to obtain the appropriate model initial conditions to contour on top of our upperair observations.

In [ ]:
# Get GFS data for contouring
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/'
                     f'Global_onedeg_ana/GFS_Global_onedeg_ana_{date:%Y%m%d}_0000.grib2')

## Make Plot

Using the declarative syntax from MetPy, we plot observations and contours on the same map panel.

There are also sqaures added to each observation location by using the axes that are generate by the declarative syntax to plot them using classic Matplotlib methods.

In [ ]:
# Add point observations
obs = PlotObs()
obs.data = data
obs.level = 500 * units.hPa
obs.time = date
obs.fields = ['temperature', 'dewpoint', 'height']
obs.locations = ['NW', 'SW', 'ENE']
obs.vector_field = ['u_wind', 'v_wind']
obs.vector_field_length = 8

# Add contours of geopotential height
cntr = ContourPlot()
cntr.data = ds.sel(lat=slice(80, 10), lon=slice(360-140, 360-40))
cntr.level = obs.level
cntr.field = 'Geopotential_height_isobaric'
cntr.clabels = True
cntr.contours = list(range(0, 10000, 60))

# set map panel features
panel = MapPanel()
panel.projection = 'lcc'
panel.area = [-125, -65, 22, 55]
panel.layers = ['states', 'borders', 'coastline']
panel.plots = [obs, cntr]

# Add map panel to figure
pc = PanelContainer()
pc.size = (20, 20)
pc.panels = [panel]

# Add a square marker at sounding site
panel.ax.scatter(data.longitude, data.latitude, s=50, marker='s', color='black', transform=ccrs.PlateCarree())

pc.show()